In [10]:
!pip install transformers datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00


In [ ]:
# use the following to add the python class files to the Colab environment
# !git clone https://<github_username>:<github_personal_access_token>@github.com/Grey-Hat-Networking/Capstone.git

In [25]:
# import created classes from python files
%autoreload 2

import os
import sys

data_path = os.path.abspath(os.path.join('.', 'Capstone/data-process'))
model_path = os.path.abspath(os.path.join('.', 'Capstone/model'))

if data_path not in sys.path:
    sys.path.append(data_path)

if model_path not in sys.path:
    sys.path.append(model_path)

from data_preprocessing_hf import EmailDatasetProcessor
from model_training import BertPhishingSequenceClassification

In [16]:
# initialize model variables

model_name = "ealvaradob/bert-finetuned-phishing"
csv_path = "Capstone/data-process/data/CEAS_08.csv"
num_extra_dims = 9 # number of numerical/categorical features to add
num_labels = 1 # unsure if correct

In [26]:
# initialize model and dataset

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(model_name)
new_model = BertPhishingSequenceClassification.from_pretrained(model_name, num_extra_dims=num_extra_dims, num_labels=num_labels)

preprocessed_data = EmailDatasetProcessor(csv_file=csv_path)

Some weights of BertPhishingSequenceClassification were not initialized from the model checkpoint at ealvaradob/bert-finetuned-phishing and are newly initialized: ['Bert.embeddings.LayerNorm.bias', 'Bert.embeddings.LayerNorm.weight', 'Bert.embeddings.position_embeddings.weight', 'Bert.embeddings.token_type_embeddings.weight', 'Bert.embeddings.word_embeddings.weight', 'Bert.encoder.layer.0.attention.output.LayerNorm.bias', 'Bert.encoder.layer.0.attention.output.LayerNorm.weight', 'Bert.encoder.layer.0.attention.output.dense.bias', 'Bert.encoder.layer.0.attention.output.dense.weight', 'Bert.encoder.layer.0.attention.self.key.bias', 'Bert.encoder.layer.0.attention.self.key.weight', 'Bert.encoder.layer.0.attention.self.query.bias', 'Bert.encoder.layer.0.attention.self.query.weight', 'Bert.encoder.layer.0.attention.self.value.bias', 'Bert.encoder.layer.0.attention.self.value.weight', 'Bert.encoder.layer.0.intermediate.dense.bias', 'Bert.encoder.layer.0.intermediate.dense.weight', 'Bert.enco

In [40]:
import datasets
from datasets import Dataset

training_dataset = Dataset.from_pandas(preprocessed_data.condensed_df)
tokenized_dataset = training_dataset.map(lambda x: tokenizer(x["body"], truncation=True))
tokenized_dataset = tokenized_dataset.cast_column("label", datasets.features.ClassLabel(num_classes=2))

Map:   0%|          | 0/117462 [00:00<?, ? examples/s]

In [50]:
# split dataset

split_data = tokenized_dataset.train_test_split(test_size=0.1)
training_data = split_data['train']
testing_data = split_data['test']

Casting the dataset:   0%|          | 0/117462 [00:00<?, ? examples/s]

ArrowInvalid: Float value 0.890282 was truncated converting to int64

In [42]:
# initialize Hugging Face Trainer

from transformers import TrainingArguments, Trainer

args = TrainingArguments(output_dir="./")
trainer = Trainer(model=new_model, train_dataset=training_data, tokenizer=tokenizer, args=args)

<ipython-input-42-325980c2fcfa>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=new_model, train_dataset=training_data, tokenizer=tokenizer, args=args)


In [43]:
# Initial evalution

trainer.evaluate(testing_data)

RuntimeError: expected scalar type Long but found Float

In [ ]:
# Run training
# See the following documentation for more details on Hugging Face Trainer:
# https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/trainer#transformers.Trainer

trainer.train()